Метод потенциалов
Ребро неориентированного графа - неупорядоченная пара вершин, называющихся концевыми вершинами ребра
Дуга - ориентированное ребро
Маршрут - чередующаяся последовательность вершин и ребер в неориентированном графе, начинающаяся и заканчивающаяся вершиной, концевыми являются вершины из еоторых одна предшествует ребру а другая следует
Цепь в графе - маршрут без повторяющихся ребер
Простая цепь - маршрут без повторяющихся вершин, за исключением, быть может(?), концевых вершин (в таком случае это простой цикл
Простой цикл - простая цепь у которой совпадают концевые вершины
Задача о потоке минимальной стоимости.
Требуется найти план перевозки продуктов при котором:
1) вся продукция из пункктов производства будет вывезена
2) Все заявки в пунктах потребления удовлетворены
3) общая стоимость перевозки минимальна
План перевозки - преписывание каждой дуге количества перевозимой по ней продукции
Потоки на дугах должны удовлетворять двум условиям:
1) для каждого пункта прозводства должно выполнятся равенство:  сумма количества всей продукции вошедшей в вершину + имеющаяся в вершине продукция должна быть равна сумме продукции вышедшей из вершины
2) для каждого пункта потребления: сумма количества всей продукции вошедшей в вершину  должна быть равна сумме продукции вышедшей из вершины + модуль потребности продуктов в пункте

In [6]:
import copy
import numpy as np
from collections import defaultdict

In [7]:
class PotentialMethod:
    def __init__(self, graph, basis):
        self.graph = copy.deepcopy(graph)
        self.basis = basis
        self.create_non_basis()
    
    def create_non_basis(self):
        self.non_basis = []
        for vertex_from, data in self.graph.items():
            vertexes_to = data[1]
            for vertex_to in vertexes_to:
                arc = (vertex_from, vertex_to)
                self.non_basis.append(arc) if arc not in self.basis else None
        print(self.non_basis)
        
    def get_cycle(self, new_arc): 
        stack = []
        colors = dict()
        extended_basis = defaultdict(list)
        for arc in self.basis:
            extended_basis[arc[0]].append(arc[1])
            extended_basis[arc[1]].append(arc[0])
        print(extended_basis)
        path = []
        for i in extended_basis.keys():
            colors[i]=0
        def dfs(v):
            
            path.append(v)
            if colors[v] == 1:
                if (path[-3] == v and not ((path[-3], path[-2]) in self.basis 
                                           and (path[-2], path[-3]) in self.basis)):
                    return False
                return True
            if colors[v] == 2: 
                return False 
            colors[v] = 1 
            for i in range(len(extended_basis[v])):
                if (extended_basis[v][i], v) in stack:
                    continue
                if dfs(extended_basis[v][i]):
                    pair = (v, extended_basis[v][i])
                    stack.append(pair) if pair in self.basis else stack.append((extended_basis[v][i], v))
                    return True
            colors[v] = 2
            return False
        
        has_cycle = dfs(new_arc[0])
        if has_cycle:
            cycle = stack
            return cycle
        return None
            
        
    def get_price(self):
        final_price = 0
        for arc in basis:
            final_price += self.graph.get(arc[0])[1].get(arc[1])[0] * self.graph.get(arc[0])[1].get(arc[1])[1]
        return final_price
        
    def solve(self):
        while True:
            M = np.zeros((len(self.graph), len(self.graph)))
            M[0][0] = 1
            C = np.array([0])
            for arc in basis:
                c = self.graph.get(arc[0])[1].get(arc[1])[0]
                C = np.append(C, [c], axis=0)
                print(arc)
                line_number = basis.index(arc) + 1
                M[line_number][arc[0] - 1] = 1            
                M[line_number][arc[1] - 1] = -1

            print(C)
            print(M)
            U = np.linalg.solve(M, C)
            print(U)
            D = {}
            for arc in self.non_basis:
                D[U[arc[0] - 1] - U[arc[1] - 1] - self.graph.get(arc[0])[1].get(arc[1])[0]] = arc
            if not D or all([key <= 0 for key in D]):
                return self.basis, self.get_price()
            print(D)
            max_delta = max(D)
            new_arc = D[max_delta]
            print(max_delta)
            print(new_arc)
            
            self.graph.get(new_arc[0])[1].get(new_arc[1])[2] = True     
            self.basis.append(new_arc)
            self.non_basis.remove(new_arc)
            
            cycle = self.get_cycle(new_arc)
            print(cycle)
            
            flows = []
            current_vertex = new_arc[0]
            for arc in cycle:
                if current_vertex == arc[0]:
                    current_vertex = arc[1]
                else:
                    current_vertex = arc[0]
                    flows.append(self.graph.get(arc[0])[1].get(arc[1])[1])
            min_flow = min(flows) if flows else 0

            current_vertex = new_arc[0]
            for arc in cycle:
                if current_vertex == arc[0]:
                    current_vertex = arc[1]
                    self.graph.get(arc[0])[1].get(arc[1])[1] += min_flow
                else:
                    current_vertex = arc[0]
                    self.graph.get(arc[0])[1].get(arc[1])[1] -= min_flow
                if not self.graph.get(arc[0])[1].get(arc[1])[1]:
                    self.basis.remove(arc)
                    self.graph.get(arc[0])[1].get(arc[1])[2] = False
                    self.non_basis.append(arc)
            print(self.graph)
                
                
        
        

        

In [8]:
intensity = 0
price = 0
flow = 0
is_basis = False
graph = {
    1: [1, {
        2: [1, 1, True]
    }],
    2: [-4, {
        6: [3, 0, False]
    }],
    3: [-5, {
        4: [5, 1, True],
        2: [3, 3, True]
    }],
    4: [-6, {
        
    }],
    5: [5, {
        3: [4, 0, False],
        4: [1, 5, True]
    }],
    6: [9, {
        1: [-2, 0, False],
        3: [3, 9, True],
        5: [4, 0, False]
    }]
}
basis = [(1, 2), (3, 2), (3, 4), (5, 4), (6, 3)]

In [9]:
solver = PotentialMethod(graph, basis)

[(2, 6), (5, 3), (6, 1), (6, 5)]


In [10]:
print(solver.solve())

(1, 2)
(3, 2)
(3, 4)
(5, 4)
(6, 3)
[0 1 3 5 1 3]
[[ 1.  0.  0.  0.  0.  0.]
 [ 1. -1.  0.  0.  0.  0.]
 [ 0. -1.  1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.]
 [ 0.  0. -1.  0.  0.  1.]]
[ 0. -1.  2. -3. -2.  5.]
{-9.0: (2, 6), -8.0: (5, 3), 7.0: (6, 1), 3.0: (6, 5)}
7.0
(6, 1)
defaultdict(<class 'list'>, {1: [2, 6], 2: [1, 3], 3: [2, 4, 6], 4: [3, 5], 5: [4], 6: [3, 1]})
[(6, 1), (1, 2), (3, 2), (6, 3)]
{1: [1, {2: [1, 4, True]}], 2: [-4, {6: [3, 0, False]}], 3: [-5, {4: [5, 1, True], 2: [3, 0, False]}], 4: [-6, {}], 5: [5, {3: [4, 0, False], 4: [1, 5, True]}], 6: [9, {1: [-2, 3, True], 3: [3, 6, True], 5: [4, 0, False]}]}
(1, 2)
(3, 4)
(5, 4)
(6, 3)
(6, 1)
[ 0  1  5  1  3 -2]
[[ 1.  0.  0.  0.  0.  0.]
 [ 1. -1.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.]
 [ 0.  0. -1.  0.  0.  1.]
 [-1.  0.  0.  0.  0.  1.]]
[  0.  -1.  -5. -10.  -9.  -2.]
{-2.0: (2, 6), -8.0: (5, 3), 3.0: (6, 5), -7.0: (3, 2)}
3.0
(6, 5)
defaultdict(<class 'list'>